# Building your first Agent step-by-step with h2oGPTe & LLM Chains

#### Description: A technical walkthrough of how to build Custom Agents with step by step with Enterprise h2oGPTe and LLM Chains

**In this blog, you will learn about LLM Chains, tools or functions calling, agents and how to leverage and customise h2oGPTe to create your first agent, all while benefiting from privately hosted LLMs, and secured data that stays with you(+)**. 

(+) as customer of enterprise h2oGPTe.

### Pre-requisites:
- Access & register https://h2ogpte.genai.h2o.ai/ 
- Python coding knowledge, virtual env installation
- Python > 3.6 installed, ability to install libraries through pip:
    - pip install h2ogpte==1.5.20
    - pip install langchain
    - pip install langchain_core
- Installed Jupyter Notebook :
    - https://docs.jupyter.org/en/latest/install/notebook-classic.html-
    - https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html

## *The basics: what are LLMs chains, tools and Agents?*

### LLM Chaining

LLM chaining or Large Language Model Chaining is the process of integrating one or multiple large language models - such as the ones hosted on h2oGPTe - with other applications, tools, and services. These chains or pipelines allow a language model to leverage the strengths of other tools and services, and to overcome its own limitations, in order to generate the most effective output possible.

A chain is like a pipeline that processes an input by using a specific combination of components: tools, LLMs, services, parsers. It can be thought of 
as a sequence of processing 'steps' that performs a certain set of operations on an input and returns the result:
For example, an LLM might be chained with an API endpoint in order to fetch real-time information on stock market or the news (... or the score of a predictive model).

The goal is to create a more powerful and versatile AI system (such as personal assistants that can provide accurate and useful outputs to a various range of inputs. 

### LLM Chaining frameworks
To date, there are several Open Source LLM Chaining frameworks : ***AutoGen, Langchain, LLamaIndex, Haystack***. Today, I will be using **Langchain** as it may be (arguably) the most flexible, versatile, and intuitive open source framework to use. 

### Tools

Tools or functions (or functions calling) are specialized functionalities that can accomplish a specific task given a set of inputs and are interfaces that an agent, chain, or LLM can use to interact with.

### Agents

Agents are systems that use LLMs as reasoning engines to decide which actions to take, tools to use and the inputs to pass them. After executing an action (like using a tool and collecting the result from the tool), the results can be fed back into the LLM to assess if further actions are required or if the process can be concluded.

**In summary**:

- Chains are sequences of processing steps for prompts. They are used within agents to define how the agent processes information.
- Tools are specialized functionalities that are used by agents or within chains for specific tasks.
- Agents are like characters with specific capabilities which use chains and tools to perform their functions.


## *what about Enterprise h2oGPTe?*

Enterprise h2oGPTe is a customisable search assistant that helps you find answers to questions about your documents, websites, or workplace content through its user interface or its python API. It is automatically able to contextualize its response with your own data (whether these are text, images, or audio) using various RAG (Retrieval-Augmented Generation) approaches or answer simply a general question. 


### Why creating your own agent with h2oGPTe?

Enterprise h2oGPTe will soon be releasing a version of its software integrating with its own agents and more functionnalities. Creating your own agent is fun way to learn how it works, when they are useful and how they may be useful your own use case or applications! In addition, it helps you to get started with h2oGPTe API using its public version (and it is free!).


## Now... Let's get started and put the theory into practise

### Connect to H2o h2oGPTe platform: Visit our [public h2oGPTe](https://h2oGPTe.genai.h2o.ai/) and follow along !

You can get started by creating [your own API key](https://h2ogpte.genai.h2o.ai/settings) using the public version of Enterprise h2oGPTe (not for production systems).
You can also check out some of the cool applications created using h2oGPTe in the background: https://genai.h2o.ai/appstore  

In [1]:
from h2ogpte import H2OGPTE

H2OGPTE_KEY ="< your api key> "

h2ogpte_client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai/',
    api_key=H2OGPTE_KEY
)

## 1 - Integrate h2oGPTe tools and functionnalities and LangChain framework

To combine h2oGPTe tools and functionnalities and LangChain framework, we will first start by wrapping our h2oGPTe client as a [LangChain Chat Model](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that uses chat messages as inputs and returns chat messages as outputs (that are not just plain text).

In [2]:
from typing import Any, AsyncIterator, Dict, Iterator, List, Optional
import json

from langchain_core.callbacks import CallbackManagerForLLMRun

from langchain_core.messages import AIMessageChunk, BaseMessage, HumanMessage, AIMessage, SystemMessage,FunctionMessage
from langchain_core.outputs import ChatResult, ChatGeneration
from langchain.schema.runnable import RunnableMap
from langchain_core.language_models import BaseChatModel
from langchain.prompts import ChatPromptTemplate


class h2ogpteChatModel(BaseChatModel):
    """h2oGPTeChatModel Class based on LangChain BaseChatModel Class.
    Example:
        .. code-block:: python

            h2ogpte = h2ogpteChatModel(h2ogpte_client = h2ogpte_client, model_name = "h2oai/h2o-danube3-4b-chat")
            result = model.invoke([HumanMessage(content="hello")])
            result = model.batch([[HumanMessage(content="hello")],
                                 [HumanMessage(content="world")]])
    """
    
    h2ogpte_client: H2OGPTE
    model_name: Optional[str]
    collection_id: Optional[str]
    kwargs: Optional[dict]

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """Pass on Prompt input to h2oGPTe via its client, create a chat session and generate outputs given this input

        Args:
            messages: the prompt composed of input messages
            stop: a list of strings on which the model should stop generating.
                  If generation stops due to a stop token, the stop token itself
                  SHOULD BE INCLUDED as part of the output. This is not enforced
                  across models right now, but it's a good practice to follow since
                  it makes it much easier to parse the output of the model
                  downstream and understand why generation stopped.
            run_manager: A run manager with callbacks for the LLM.
        """
        
        prompt = ChatPromptTemplate(messages=messages)
        messages_as_string = prompt.format()
        if not self.model_name:
            self.model_name = self.h2ogpte_client.get_llms()[0]['display_name']
            
        if not self.collection_id:
            self.collection_id = None
        
        chat_session_id = h2ogpte_client.create_chat_session(self.collection_id)
        with h2ogpte_client.connect(chat_session_id) as session:
            response = session.query(message = messages_as_string,
                                         llm = self.model_name, 
                                         **kwargs).content
                
            responseAIMessage = AIMessage(
                                    content=response,
                                    additional_kwargs={},
                                    response_metadata={},
                                )

        return ChatResult(generations=[ChatGeneration(message=responseAIMessage)])
    
    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model.
        Used to uniquely identify the type of the model. Used for logging."""
        return self.model_name

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters.
        Represent model parameterization for tracing purposes.
        """
        return [model for model in self.h2ogpte_client.get_llms() if model["display_name"] == self.model_name][0]

h2ogpteChatModel takes as input an instance of h2oGPTe client, an optionally the llm *model_name* the user would like to pick from h2ogpte as well as the *collection_ID* or string ID of the collection of documents to chat with using Retrieval Augmented Generation (RAG). In parallel, Enterprise h2oGPTe has a highly customizable prompting to talk to LLM, document & collection of documents, summarise and extract information and is...  LLM agnostic - you can choose the model you need for your use case, including your own fine tuned model! 

The *_generate* method is used to generate a chat result from a prompt: we create a chat session with h2ogpte, pass the user prompt (provide additional optional arguments) and we return a ChatResult of the response generated from h2ogpte chat session as AIMessage.

One of the model we can use is our own H2o series of small language models (H2O-Danube3-4B & H2O-Danube3-500M), fined-tuned for conversation using H2O LLM Studio. 

In [3]:
model_name = None

if "h2oai/h2o-danube3-4b-chat" in [llm['display_name'] for llm in h2ogpte_client.get_llms()]:
    model_name = 'h2oai/h2o-danube3-4b-chat'
model_name

'h2oai/h2o-danube3-4b-chat'

<!-- ## TODO WORD ON DANUBE -->

In [4]:
h2ogpteChat = h2ogpteChatModel(h2ogpte_client = h2ogpte_client, model_name = model_name)

In addition to the model, we can include parameters of the RAG, and the LLM arguments such as temperature:

In [5]:
h2ogpteChat.invoke("whatsup", llm_args = {"temperature": 0})

AIMessage(content="Hello! I'm here to help you with any questions or information you need. How can I assist you today?", id='run-5c745867-13bd-4904-a6da-cd51bc7f766c-0')

In [6]:
h2ogpteChat.invoke("whatsup", llm_args = {"temperature": 1})

AIMessage(content="Interesting question! As an AI, I don't have feelings, but I'm here to help answer your questions and provide information in ways that I'm programmed to do.", id='run-47cd58fa-a73d-4186-ab87-357b04fff30c-0')

##
#### Now, we have created our first component of an LLM Chain (h2ogpteChatModel), let's create our first chain with "Langchain Expression language" or LCEL.

To do so, we will start by adding to our chat Model one of the simplest component of a chain: Prompt template. it will help to format the user input and provides a consistent and standardized way to present the prompt to the LLM.

Please note, h2oGPTe has its own prompt template catalog, that you can augment with your own defined template for your use case. Find out more [here](https://docs.h2o.ai/enterprise-h2ogpte/blog/tags/v-1-4-13#introducing-the-prompt-catalog).

Next, the prompt is then passed to the LLM component of the chain: h2oGPTe Chat which will processe the input prompt and generates a response accordingly:

In [7]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "My name is Audrey, {input}"
)
chain = prompt | h2ogpteChat

chain.invoke({"input":"hello, how are you?"})

AIMessage(content="Hello Audrey! I'm an AI assistant designed to help answer your questions and provide information. How can I assist you today?", id='run-0e6f9eea-b6c4-45ce-ad9a-22ce9df864b1-0')


That's it ! very easy, we have created our first chain!

##
Now, depending on the requirements and objectives of the downstream application this chain is used for, the response from h2ogpteChat - the last step on our current chain - can be displayed to the user, further processed, or fed into the next component in the chain. 

#### Let's add a simple String Output parser to our chain:

In [8]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt | h2ogpteChat | StrOutputParser()

chain.invoke({"input":"hello, how are you?"})

"Hello Audrey! I'm an AI assistant designed to help answer your questions and provide information. How can I assist you today?"


#### Now, would it not be great if our h2ogpteChat model could interact with user defined tools, APIs to fetch some data, perform some actions whenever the user or an application is requesting it?

This is possible by binding functions or tools to our h2ogpteChat model to choose from. 

First, we need to declare our functions definition. Similarly to what can be found in function dosctrings, we need to describe our function in a json schema: description, parameters that are required to call the function along with their types and description. 


Let's say we have two functions called *get_country_information* and *get_current_weather* defined as follow:

In [9]:
functions = [
        {
            "name": "get_country_information",
            "description": "the function get_country_information can be used to Get information about a country such as: its capital, currency, population or maps",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "The country of interest, for example Italy",
                    },
                    "field_to_extract": {"type": "string", "enum": ["capital", "currency", "population", "maps"]},
                },
                "required": ["country", "field_to_extract"],
            },
        },
    
    {
            "name": "get_current_weather",
            "description": "the function get_current_weather can be used to get the current temperature in Celsius or Fahrenheit in a given location (City, State)",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },

  ]
functions

[{'name': 'get_country_information',
  'description': 'the function get_country_information can be used to Get information about a country such as: its capital, currency, population or maps',
  'parameters': {'type': 'object',
   'properties': {'country': {'type': 'string',
     'description': 'The country of interest, for example Italy'},
    'field_to_extract': {'type': 'string',
     'enum': ['capital', 'currency', 'population', 'maps']}},
   'required': ['country', 'field_to_extract']}},
 {'name': 'get_current_weather',
  'description': 'the function get_current_weather can be used to get the current temperature in Celsius or Fahrenheit in a given location (City, State)',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city and state, e.g. San Francisco, CA'},
    'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
   'required': ['location']}}]

##

By binding the functions defined above to h2ogpteChat model class, we are letting it know what functions we have access to each time the model is invoked, what they are for and how to appropriately invoke them using this json schema. This way, the LLM can determine based on the user query to invoke those tools if necessary.

Let's see now on a working example how we achieve that.

In [10]:
h2ogpteChat_bind = h2ogpteChat.bind(functions=functions)

In [11]:
h2ogpteChat

h2ogpteChatModel(h2ogpte_client=<h2ogpte.h2ogpte.H2OGPTE object at 0x7fe200704790>, model_name='h2oai/h2o-danube3-4b-chat')

In [12]:
h2ogpteChat_bind

RunnableBinding(bound=h2ogpteChatModel(h2ogpte_client=<h2ogpte.h2ogpte.H2OGPTE object at 0x7fe200704790>, model_name='h2oai/h2o-danube3-4b-chat'), kwargs={'functions': [{'name': 'get_country_information', 'description': 'the function get_country_information can be used to Get information about a country such as: its capital, currency, population or maps', 'parameters': {'type': 'object', 'properties': {'country': {'type': 'string', 'description': 'The country of interest, for example Italy'}, 'field_to_extract': {'type': 'string', 'enum': ['capital', 'currency', 'population', 'maps']}}, 'required': ['country', 'field_to_extract']}}, {'name': 'get_current_weather', 'description': 'the function get_current_weather can be used to get the current temperature in Celsius or Fahrenheit in a given location (City, State)', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state, e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum'

### Modifying the h2ogpteChatModel class to add h2oGPTe guided generation for Function calling
Now, we just need to give some instructions to our h2ogpteChat model to:

- make it aware of the functions available
- assess when and if these functions are necessary given an input prompt
- respond in a consistent manner that will allow downstream to call the function appropriately (in our case convert natural language into valid API calls)

For this, we are adding the *_function_calling_prompt* property:

In [ ]:
    @property
    def _function_calling_prompt(self) -> str:
        """Get the function calling prompt for the LLM"""
        return  """ // namespace functions
        You serve as a wrapper for utilizing multiple tools. Each tool that can be used MUST be specified in the list of the namespace functions section above ONLY. Ensure that the parameters provided to each tool are valid according to that tool's specification. Only functions in the functions namespace are permitted, 
        DO NOT GUESS or MAKE UP a function if it is not available in the list of namespace functions section above.
        Their description, properties are described. Given the query given below, respond with the function names to use, list of arguments and their value in a valid json. 
        For example, if the instruction are matching one or more of the description of the function above:

        {'content': '', 'additional_kwargs': {'function_call': {'name': '<name of relevant listed function_name>', 'arguments': '{"<name of argument_1 of listed function>": "<argument 1 value>", "<name of argument_2 of listed function>": "<argument 2 value>"}' }}}

        if none of the functions help with the query, are matching or relevant to the query/instructions below, DO NOT include the functions and simply respond instead in the below format to the best of your knowledge of the question in a valid json:

        {'content': '<response>', 'additional_kwargs': {}}
        
        here is the query, please respond:
        """

We augment the *_generate* method to use *_function_calling_prompt*, check if a function call is relevant and leverage h2ogpte [guided generation](https://docs.h2o.ai/enterprise-h2ogpte/blog/v1.5#guided-generation) by providing the json schema we require the h2ogpteChat model to return IF a function call is deemed necessary given a user input:

<!-- ## TODO WORD ON GUIDED GENERATION -->

In [ ]:
    if 'functions' in kwargs:
        requires_function = session.query(message = messages_as_string,
                                                     system_prompt = f"If Only the following functions are available: {str(kwargs['functions'])}, do any of the listed functions are relevant in answering entirely the following user query? Respond ONLY with True or False. user query:",
                                 llm = self.model_name).content
        if "true" in str.lower(requires_function):
            function_calling_prompt = f"""## functions: \n namespace functions \\ \n {str(kwargs["functions"])} """ + self._function_calling_prompt
            response = session.query(message = messages_as_string,
                                     system_prompt = function_calling_prompt,
                                     llm = self.model_name,
                                    ).content
            response = session.query(message = f'''
                                    Return a valid JSON
                                    Escape any backslasheswith a backslash.
                                    Escape any double quotes with \\.
                                    Escape any newline characters so that they become \\n.
                                    The response must be a valid JSON. 

                                    Please review : {response}
                                     ''',
                                     system_prompt = function_calling_prompt,
                                     llm = self.model_name,
                                     llm_args=dict(
                                        response_format='json_object',
                                        guided_json=self.gjson
                                 )).content

Here is the final Result:

In [13]:
class h2ogpteChatModel(BaseChatModel):
    """A custom chat model that augment LangChain BaseChatModel Class.
    Example:

        .. code-block:: python

            h2ogpte = h2ogpteChatModel(h2ogpte_client = h2ogpte_client, model_name = "h2oai/h2o-danube3-4b-chat")
            result = model.invoke([HumanMessage(content="hello")])
            result = model.batch([[HumanMessage(content="hello")],
                                 [HumanMessage(content="world")]])
    """
    
    h2ogpte_client: H2OGPTE
    "h2ogpte client to connect to Enterprise h2oGPTe and tools "
    model_name: Optional[str]
    collection_id: Optional[str]
    gjson: Optional[dict] = {
                             "$schema": "http://json-schema.org/draft-07/schema#",
                             "type": "object",
                              "properties": {
                                "content": {
                                  "type": "string"
                                },
                                "additional_kwargs": {
                                  "type": "object",
                                  "default": {},
                                  "properties": {
                                    "function_call": {
                                     "type": "object",
                                     "default": {},
                                    "properties": {
                                           "name": {
                                             "type": "string"
                                            },
                                            "arguments": {
                                                "type": "string",
                                            }
                                        }
                                    }
                                },
                                "required": [] 
                                }
                              },
                              "required": [ "content", "additional_kwargs"]
                            }
    kwargs: Optional[dict]

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """Pass on Prompt input to h2oGPTe client to creat a chat session and generate outputs given an input.

        Args:
            messages: the prompt composed of input messages
            stop: a list of strings on which the model should stop generating.
                  If generation stops due to a stop token, the stop token itself
                  SHOULD BE INCLUDED as part of the output. This is not enforced
                  across models right now, but it's a good practice to follow since
                  it makes it much easier to parse the output of the model
                  downstream and understand why generation stopped.
            run_manager: A run manager with callbacks for the LLM.
        """

        prompt = ChatPromptTemplate(messages=messages)
        messages_as_string = prompt.format()

        if not self.model_name:
            self.model_name = self.h2ogpte_client.get_llms()[0]['display_name']
        if not self.collection_id:
            self.collection_id = None
        
        chat_session_id = h2ogpte_client.create_chat_session(self.collection_id)
        with h2ogpte_client.connect(chat_session_id) as session:
            if 'functions' in kwargs:
                
                is_answered="False"
                if len(messages)>1:
                    is_answered = session.query(message = prompt[1:].format(),
                                                system_prompt = f"Is the response to << {messages[0].content} >> been answered by the below messages only, answer with True or False:",
                                                llm = self.model_name).content
                    
                requires_function = session.query(message = messages_as_string,
                                                            system_prompt = f"Only the following functions are available: {str(kwargs['functions'])}, are any of the listed functions relevant in answering entirely the following user query? Respond ONLY with True if relevants, or False if not. user query:",
                                                            llm = self.model_name).content
                
                if "true" in str.lower(requires_function) and "false" in str.lower(is_answered):
                    function_calling_prompt = f"""## functions: \n namespace functions // \n {str(kwargs["functions"])} """ + self._function_calling_prompt
                    response = session.query(message = messages_as_string,
                                             system_prompt = function_calling_prompt,
                                             llm = self.model_name,
                                            ).content
                    
                    #  guided json/generation
                    response = session.query(message = f'''
                                            Return a valid JSON
                                            Escape any backslashes with a backslash.
                                            The response must be a valid JSON. 
                                            
                                            Please review : {response}
                                             ''',
                                             system_prompt = function_calling_prompt,
                                             llm = self.model_name,
                                             llm_args=dict(
                                                response_format='json_object',
                                                guided_json=self.gjson
                                             )).content
                    try:
                        json_response = json.loads(response)
                        if json_response['additional_kwargs']['function_call']['name'] not in [fct['name'] for fct in functions]:
                            json_response['additional_kwargs'] = {}

                        responseAIMessage = AIMessage(
                            content=json_response['content'],
                            additional_kwargs=json_response['additional_kwargs'],
                            response_metadata={},
                        )
                    except:
                        print("failed to extract response['content']")
                        responseAIMessage = AIMessage(
                            content=response,
                            additional_kwargs={},
                            response_metadata={},
                        )
                elif "true" in str.lower(is_answered):
                    responseAIMessage = AIMessage(content=[message.content for message in messages if message.type=="function"][-1],)
                else:
                    kwargs.pop("functions", None)
                    response = session.query(messages_as_string,
                                             llm = self.model_name, 
                                             **kwargs
                                            ).content
                    responseAIMessage = AIMessage(
                                        content=response,
                                        additional_kwargs={},
                                        response_metadata={},
                                    )
            else:
                kwargs.pop("functions", None)
                response = session.query(messages_as_string,
                                         llm = self.model_name, **kwargs
                                        ).content
                responseAIMessage = AIMessage(
                                    content=response,
                                    additional_kwargs={},
                                    response_metadata={},
                                )
        generation = ChatGeneration(message=responseAIMessage)
        return ChatResult(generations=[generation])

    
    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model.
        Used to uniquely identify the type of the model. Used for logging."""
        return self.model_name

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters.
        Represent model parameterization for tracing purposes.
        """
        return [model for model in self.h2ogpte_client.get_llms() if model["display_name"] == self.model_name][0]
    @property
    def _function_calling_prompt(self) -> str:
        """Get the function calling prompt for the LLM"""
        return  """ // namespace functions
        You serve as a wrapper for utilizing multiple tools. Each tool that can be used MUST be specified in the list of the namespace functions section above ONLY. Ensure that the parameters provided to each tool are valid according to that tool's specification. Only functions in the functions namespace are permitted, 
        DO NOT GUESS or MAKE UP a function if it is not available in the list of namespace functions section above.
        Their description, properties are described. Given the query given below, respond with the function names to use, list of arguments and their value in a valid json. 
        For example, if the instruction are matching one or more of the description of the function above:

        {'content': '', 'additional_kwargs': {'function_call': {'name': '<name of relevant listed function_name>', 'arguments': '{"<name of argument_1 of listed function>": "<argument 1 value>", "<name of argument_2 of listed function>": "<argument 2 value>"}' }}}

        if none of the functions help with the query, are matching or relevant to the query/instructions below, DO NOT include the functions and simply respond instead in the below format to the best of your knowledge of the question in a valid json:

        {'content': '<response>', 'additional_kwargs': {}}
        
        here is the query, please respond:
        """

In [14]:
h2ogpteChat = h2ogpteChatModel(h2ogpte_client = h2ogpte_client, model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1')

In [15]:
h2ogpteChat_bind = h2ogpteChat.bind(functions=functions)

In [16]:
h2ogpteChat.invoke("how are you?")

AIMessage(content="I'm an AI model, so I don't have feelings, but I'm here and ready to help you with any questions or problems you have to the best of my ability! How can I assist you today?", id='run-716e957d-8de6-4f9b-80ed-28a41c67c43d-0')

In [17]:
h2ogpteChat_bind.invoke("how are you?")

AIMessage(content="I'm an AI model, so I don't have feelings, but I'm here and ready to help you with any questions or problems you have to the best of my ability! How can I assist you today?", id='run-364f791f-3499-4fee-9de0-024c22141164-0')

In [18]:
prompt

ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='My name is Audrey, {input}'))])

In [19]:
chain = prompt | h2ogpteChat_bind

In [20]:
chain.invoke({"input": "how much money do you need to earn to live comfortably in San Francisco?"})

AIMessage(content="Hello Audrey, I'm h2oGPTe. The cost of living can vary greatly depending on individual lifestyle, but on average, to live comfortably in San Francisco, it's often suggested that you might need a salary of around $120,000 to $130,000 per year. This takes into account the high cost of housing, healthcare, transportation, and other expenses. However, please note that this is a general estimate and individual needs can vary.", id='run-500eb15a-fc13-41ca-862f-1068c4542392-0')

Here the h2ogpteChatModel deemed based on my instrutions that a function call was not necessary and used its own internal knowledge to generate a general answer to my question. 

*What if I ask questions relevant to the functions I declare and bind to my h2ogpteChatModel?*

In [21]:
chain.invoke({"input": "hello, what is the temperature in San Francisco?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "San Francisco, CA", "unit": "fahrenheit"}'}}, id='run-91276a03-63ad-4bae-b928-68d8b08730c1-0')

In [22]:
chain.invoke({"input": "what is the number of inhabitants in Albania?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_country_information', 'arguments': '{"country": "Albania", "field_to_extract": "population"}'}}, id='run-0d1b32c4-02b3-47b5-a511-e16fe0a3a3f9-0')

In [23]:
chain.invoke({"input": "what is the currency in Turkey?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_country_information', 'arguments': '{"country": "Turkey", "field_to_extract": "currency"}'}}, id='run-b84a82fc-add8-4bfc-b1eb-4f3ca2680f83-0')

Now the outputs are consisting of the argument *additional_kwargs* which contains the relevant function name, arguments and associated value, that is, all the informations necessary to call the function required to obtain the answer to the user query!

### Great, now let's go a step further so we can execute the underlying relevant function and return the output to the user when relevant.

In [24]:
import requests
from typing import Literal
from pydantic.v1 import BaseModel, Field
import datetime
from langchain.agents import tool

# Define the input schema
class OpenCountryInfo(BaseModel):
    country: str = Field(..., description="Name of the country to fetch data for. For example 'Italy'")
    field_to_extract: Literal['capital', 'currency', 'population', 'maps'] = Field(..., description="Name of the field or information to extract data about")
    
@tool(args_schema=OpenCountryInfo)
def get_country_information(country:str, field_to_extract:str) -> dict:
    """this function is used to extract information about a country such as: capital, currency, population OR maps"""
    
    BASE_URL = f"https://restcountries.com/v3.1/name/{country}/?fields={field_to_extract}"
    response = requests.get(BASE_URL)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")
    return f"{country}'s {field_to_extract} is {list(results[0].values())[0]}"

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [25]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [
    convert_to_openai_function(f) for f in [
        get_current_temperature,
        get_country_information,
    ]
]

In [26]:
functions

[{'name': 'get_current_temperature',
  'description': 'Fetch current temperature for given coordinates.',
  'parameters': {'type': 'object',
   'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
     'type': 'number'},
    'longitude': {'description': 'Longitude of the location to fetch weather data for',
     'type': 'number'}},
   'required': ['latitude', 'longitude']}},
 {'name': 'get_country_information',
  'description': 'this function is used to extract information about a country such as: capital, currency, population OR maps',
  'parameters': {'type': 'object',
   'properties': {'country': {'description': "Name of the country to fetch data for. For example 'Italy'",
     'type': 'string'},
    'field_to_extract': {'description': 'Name of the field or information to extract data about',
     'enum': ['capital', 'currency', 'population', 'maps'],
     'type': 'string'}},
   'required': ['country', 'field_to_extract']}}]

### Great, now let's go a step further so we can execute the underlying relevant function and return the output to the user when relevant.

First, we will be borrowing the [OpenAIFunctionsAgentOutputParser](https://api.python.langchain.com/en/latest/_modules/langchain/agents/output_parsers/openai_functions.html#OpenAIFunctionsAgentOutputParser), which simply check if the output from our h2ogpteChat Model contains a *function_call* argument or not and define wether an action such as executing a function call is required from the agent (***AgentAction***) or the agent job is done here (***AgentFinish***) .

In [27]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.agents import AgentAction, AgentActionMessageLog, AgentFinish

In [28]:
h2ogpteChat = h2ogpteChatModel(h2ogpte_client = h2ogpte_client, model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1')
h2ogpteChat_bind = h2ogpteChat.bind(functions=functions)

now we can define a route function, which will check if the agent has reached a final (*AgentFinish*) status and return the final answer or will route the query towards the appropriate tools (*result.tool*) and execute it with the function call arguments (*result.tool_input*):

In [46]:
from langchain.schema.agent import AgentFinish

def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "get_current_temperature": get_current_temperature,
            "get_country_information": get_country_information,
        }
        print(result.tool_input) # When using a tool, let's print the input / arguments of the tool
        print(result.tool) 
        print(tools[result.tool].run(result.tool_input))
        return tools[result.tool].run(result.tool_input) 

In [47]:
chain = prompt | h2ogpteChat_bind | OpenAIFunctionsAgentOutputParser() | route

In [31]:
result = chain.invoke({"input": "Give me map of Spain?"})

{'country': 'Spain', 'field_to_extract': 'maps'}
get_country_information
Spain's maps is {'googleMaps': 'https://goo.gl/maps/138JaXW8EZzRVitY9', 'openStreetMaps': 'https://www.openstreetmap.org/relation/1311341'}


In [32]:
result

"Spain's maps is {'googleMaps': 'https://goo.gl/maps/138JaXW8EZzRVitY9', 'openStreetMaps': 'https://www.openstreetmap.org/relation/1311341'}"

In [33]:
result = chain.invoke({"input": "hello, what is the temperature in San Francisco, CA?"}) 
result

{'latitude': 37.7749, 'longitude': -122.4194}
get_current_temperature
The current temperature is 18.9°C


'The current temperature is 18.9°C'

In [34]:
result = chain.invoke({"input": "What is the number of inhabitants in Hungary?"})

{'country': 'Hungary', 'field_to_extract': 'population'}
get_country_information
Hungary's population is 9749763


In [35]:
result

"Hungary's population is 9749763"

<!-- ## TODO WORD ON GUARDRAILS -->

We are almost there! 

Let's add some messages placeholder, that is a placeholder for intermediary steps or scratchpad that should be a sequence of messages that contains the previous agent tool invocations and the corresponding tool outputs put together. This will be useful to further enhance our agent in using previous messages, including initial user query and the intermediary steps.

<!-- ## TODO MessagesPlaceholder explanation -->

In [36]:
from langchain_core.prompts.chat import HumanMessagePromptTemplate
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor


prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("My name is Audrey, {input}"),
    MessagesPlaceholder(variable_name="intermediate_steps"),
])

First, we will be borrowing the [format_to_openai_function_messages](https://api.python.langchain.com/en/latest/_modules/langchain/agents/format_scratchpad/openai_functions.html#format_to_openai_function_messages), which simply 
send to the LLM the Steps the LLM has taken to date, along with observations and returns a list of messages to send to the LLM for the next prediction. For this we also sligtly modified the h2ogpteChat Model class to check if the initial query has been answered or not yet.

In [37]:
from langchain.agents.format_scratchpad.openai_functions import format_to_openai_function_messages
from langchain.schema.runnable import RunnablePassthrough 


agent_chain = RunnablePassthrough.assign(
    intermediate_steps= lambda x: format_to_openai_function_messages(x["intermediate_steps"])
) | prompt | h2ogpteChat_bind | OpenAIFunctionsAgentOutputParser()

In [38]:
agent_executor = AgentExecutor(agent=agent_chain, 
                               tools=[get_country_information, get_current_temperature], 
                               verbose=True, 
                               handle_parsing_errors=True)

In [39]:
agent_executor.invoke({"input": "what is the current temperature in Istres, France in degrees celcius?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': '43.5333', 'longitude': '5.0833'}`


The current temperature is 20.6°CThe current temperature is 20.6°C

> Finished chain.


{'input': 'what is the current temperature in Istres, France in degrees celcius?',
 'output': 'The current temperature is 20.6°C'}

In [40]:
agent_executor.invoke({"input": "Give me a map of Hungary?"})



> Entering new AgentExecutor chain...

Invoking: `get_country_information` with `{'country': 'Hungary', 'field_to_extract': 'maps'}`


Hungary's maps is {'googleMaps': 'https://goo.gl/maps/9gfPupm5bffixiFJ6', 'openStreetMaps': 'https://www.openstreetmap.org/relation/21335'}Hungary's maps is {'googleMaps': 'https://goo.gl/maps/9gfPupm5bffixiFJ6', 'openStreetMaps': 'https://www.openstreetmap.org/relation/21335'}

> Finished chain.


{'input': 'Give me a map of Hungary?',
 'output': "Hungary's maps is {'googleMaps': 'https://goo.gl/maps/9gfPupm5bffixiFJ6', 'openStreetMaps': 'https://www.openstreetmap.org/relation/21335'}"}

In [45]:
agent_executor.invoke({"input": "what is my name in chinese"})



> Entering new AgentExecutor chain...
Hello Audrey, in simplified Chinese your name would be 奥德里, pronounced as "Aòdélǐ". Please note that Chinese names can have many variations and this is one of them.

> Finished chain.


{'input': 'what is my name in chinese',
 'output': 'Hello Audrey, in simplified Chinese your name would be 奥德里, pronounced as "Aòdélǐ". Please note that Chinese names can have many variations and this is one of them.'}

##

Yay! we created an agent wiht h2ogpte and Langchain!

##

Now, obviously this is a very simple agent and if you wanted go a step further, you can try and improve:

- error handling: for example if the function fails due to unavailability or invalid arguments
- Handling simulatenous or sequential function call: Modify the h2ogpteChat model to handle intermediary steps that require multiple function call for more complex user query
- Handling Memory: enabling the agent to keep a chat history for follow up questions
- Prompt engineering: test out different Instructions or Few-shot learning examples to enforce a model behavior through all conversations and messages
- Guardrails: enable h2ogpte guardrails and create your own! 


##
##
#### Reference sources: h2oGPTe, Langchain, DeepLearning.ai, Medium Blog ressources


- https://h2ogpte.genai.h2o.ai/
- https://docs.h2o.ai/enterprise-h2ogpte/
- https://docs.h2o.ai/enterprise-h2ogpte/get-started/use-cases

- https://blog.spheron.network/a-comprehensive-comparison-of-llm-chaining-frameworks
- https://python.langchain.com/v0.2/docs/how_to/custom_chat_model/
- https://learn.deeplearning.ai/courses/
